## Interacting with the IBM Watson Natural Language Understanding API; POST vs GET

Another useful API, especially when dealing with text, is the [IBM Watson  Natural Language Understanding API](https://console.bluemix.net/catalog/services/natural-language-understanding), which offers a variety of text analysis functionalities, such as sentiment analysis, entity extraction, keyword extraction, etc.



#### /analyze call

We will first start with the `GET /analyze` API call ([documentation](https://www.ibm.com/watson/developercloud/natural-language-understanding/api/v1/#get-analyze)), which takes as input a piece of text, and returns an analysis across various dimensions.

The call below gets as input a "text" variable, and returns back the sentiment of the text.

In [11]:
import requests
import json

def getSentiment(text):
    endpoint = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"

    # You can register and get your own credentials
    # The ones below have a quota of 1000 calls per day 
    # and can run out quickly if multiple people use these
    username = "XXXXXXXXXXXXX"
    password = "XXXXXXXXXXXX"
    
    parameters = {
        #'features' : 'concepts,categories,emotion,entities,keywords,relations,semantic_roles,sentiment',
        'features': 'emotion,sentiment,keywords',
        'version' : '2017-02-27',
        'text': text,
        'language' : 'en',
        # url = url_to_analyze, this is an alternative to sending the text
    }

    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    
    return resp.json()

In [16]:
text = '''
SAN FRANCISCO — A former engineer at Uber sued the company on Monday, 
claiming that co-workers sexually harassed her during her time at the 
ride-hailing service and that its human resources department failed to 
act on her complaints.

In the lawsuit, Ingrid Avendaño, who joined Uber in 2014 and left last 
year, also said that despite receiving praise for her work, she was denied 
promotions or pay increases in retaliation for reporting misconduct at the company.

Ms. Avendaño said many of her complaints went ignored. In one incident, 
she said, she told the human resources department about a male software 
engineer who repeatedly said at a recruiting event that Uber was the 
“type of company where women can sleep their way to the top.”
'''

data = getSentiment(text)

In [17]:
data

{'emotion': {'document': {'emotion': {'anger': 0.408944,
    'disgust': 0.328995,
    'fear': 0.013541,
    'joy': 0.021606,
    'sadness': 0.543152}}},
 'keywords': [{'relevance': 0.989368, 'text': 'human resources department'},
  {'relevance': 0.652039, 'text': 'Uber'},
  {'relevance': 0.606341, 'text': 'SAN FRANCISCO'},
  {'relevance': 0.585027, 'text': 'Ingrid Avendaño'},
  {'relevance': 0.580584, 'text': 'ride-hailing service'},
  {'relevance': 0.555364, 'text': 'recruiting event'},
  {'relevance': 0.546483, 'text': 'male software'},
  {'relevance': 0.544234, 'text': 'Ms. Avendaño'},
  {'relevance': 0.474297, 'text': 'complaints'},
  {'relevance': 0.453179, 'text': 'engineer'},
  {'relevance': 0.44764, 'text': 'company'},
  {'relevance': 0.401373, 'text': 'co-workers'},
  {'relevance': 0.38624, 'text': 'retaliation'},
  {'relevance': 0.371092, 'text': 'misconduct'},
  {'relevance': 0.370544, 'text': 'increases'},
  {'relevance': 0.369375, 'text': 'lawsuit'},
  {'relevance': 0.3671

In [18]:
data['sentiment']

{'document': {'label': 'negative', 'score': -0.576715}}

In [19]:
data['emotion']

{'document': {'emotion': {'anger': 0.408944,
   'disgust': 0.328995,
   'fear': 0.013541,
   'joy': 0.021606,
   'sadness': 0.543152}}}

#### Entities call

[Full Documentation of the call](https://www.ibm.com/watson/developercloud/natural-language-understanding/api/v1/#entities)

This is a an API call that extracts entities from the text, and also the sentiment and emotion for each of these entities. You will also see that there is the capability of "normalizing" each entity, so that two different ways of saying the same thing get mapped to the same entity. So for example, "President Trump" and "Donald Trump" get mapped to the same Knowledge Graph entity.

In [20]:
import requests
import json

def processURL(url_to_analyze):
    endpoint_watson = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"
    params = {
        'version': '2017-02-27',
    }
    headers = { 
        'Content-Type': 'application/json',
    }
    watson_options = {
      "url": url_to_analyze,
      "features": {
        "entities": {
          "sentiment": True,
          "emotion": True,
          "limit": 10
        }
      }
    }
    username = "XXXXXXXXXXXXX"
    password = "XXXXXXXXXXXX"

    resp = requests.post(endpoint_watson, 
                         data=json.dumps(watson_options), 
                         headers=headers, 
                         params=params, 
                         auth=(username, password) 
                        )
    return resp.json()


url_to_analyze = 'https://www.nbcnews.com/politics/politics-news/jeff-sessions-remarks-anglo-american-heritage-law-enforcement-n847286'

data = processURL(url_to_analyze)

In [21]:
# Let's see what we get back as top-level attributes
data.keys()

dict_keys(['entities', 'usage', 'language', 'retrieved_url'])

In [22]:
# Let' see the entities list
data["entities"]

[{'count': 4,
  'emotion': {'anger': 0.398578,
   'disgust': 0.344209,
   'fear': 0.009659,
   'joy': 0.250321,
   'sadness': 0.206066},
  'relevance': 0.84073,
  'sentiment': {'label': 'negative', 'score': -0.454001},
  'text': 'Justice Department',
  'type': 'Organization'},
 {'count': 1,
  'emotion': {'anger': 0.0,
   'disgust': 0.0,
   'fear': 0.0,
   'joy': 0.0,
   'sadness': 0.0},
  'relevance': 0.500823,
  'sentiment': {'label': 'neutral', 'score': 0.0},
  'text': "National Sheriffs' Association",
  'type': 'Organization'},
 {'count': 2,
  'emotion': {'anger': 0.389789,
   'disgust': 0.202735,
   'fear': 0.07119,
   'joy': 0.036202,
   'sadness': 0.434254},
  'relevance': 0.48982,
  'sentiment': {'label': 'negative', 'score': -0.411921},
  'text': 'attorney',
  'type': 'JobTitle'},
 {'count': 2,
  'disambiguation': {'dbpedia_resource': 'http://dbpedia.org/resource/England',
   'name': 'England',
   'subtype': ['PoliticalDistrict',
    'AdministrativeDivision',
    'GovernmentalJ

In [23]:
# Let' see the first entity. Notice the "disambiguated" attribute that
# points to "canonical" versions of the entity, in DBPedia, Freebase, OpenCYC, YAGO, etc
data["entities"][0]

{'count': 4,
 'emotion': {'anger': 0.398578,
  'disgust': 0.344209,
  'fear': 0.009659,
  'joy': 0.250321,
  'sadness': 0.206066},
 'relevance': 0.84073,
 'sentiment': {'label': 'negative', 'score': -0.454001},
 'text': 'Justice Department',
 'type': 'Organization'}

In [24]:
# This function takes as input the result
# from the IBM Watson API and returns a list
# of entities that are relevant (above threshold)
# to the article
def getEntities(data, threshold):
    result = []
    for entity in data["entities"]:
        relevance = float(entity['relevance'])
        if relevance > threshold:
            result.append(entity['text'])
    return result

getEntities(data, 0.25)

['Justice Department',
 "National Sheriffs' Association",
 'attorney',
 'England',
 'Coretta Scott King',
 'Bernice King',
 'Ku Klux Klan',
 'Martin Luther King Jr.']

#### Exercise

* First of all, **get your own credentials for the IBM Watson API**. The demo key that we use above has a limited quota.
* Use an API to get news articles. 
    * Option 1: Use the NY Times API to fetch the Top Stories News. You can register and get an API key at https://developer.nytimes.com/. The `Top Stories V2 API` provides the details of the news of the day: (The API call documentation is at https://developer.nytimes.com/top_stories_v2.json and the API Call is  https://api.nytimes.com/svc/topstories/v2/home.json?api-key=PUTYOURKEYHERE). Repeat the entity extraction process from above.
    * Option 2: Use the API at https://newsapi.org to fetch the news from various sources. Print the entities that are currently being discussed in the news, together with their relevance value and the associated sentiment.
    * Option 3: Use the Guardian API at https://open-platform.theguardian.com/documentation/ to fetch news from The Guardian.


### Exercise: Using the Spotify API

We will now use the Spotify API to get information about an artist. The documentation of the calls is at https://developer.spotify.com/web-api/endpoint-reference/. For now, use only the calls that do not require an OAuth authentication. 

Tasks:
* We can first find the id of an artist using the `/v1/search?type=artist` API call. The documentation of the `search-item` endpoint is at https://developer.spotify.com/web-api/search-item/.
* Once you get back the ID of the artist, use the `get artist` endpoint, to get further information about the artist: https://developer.spotify.com/web-api/get-artist/
* Study the documentation and figure out how to get the albums of an article, the top tracks for an artist, and the related artists.



